In [13]:
import json
import numpy as np
import numpy.ma as ma
import pprint

def initProgram(file_name):
    # открыть файл с данными, поместить все данные в переменную raw_data
    with open(file_name) as json_file:
        raw_data = json.load(json_file)
    
    # подготовка данных
    suppliers = np.array(raw_data['Suppliers'], dtype=int)
    consumers = np.array(raw_data['Consumers'], dtype=int)
    costs = np.array(raw_data['Costs'], dtype=int)
    capacity = np.zeros(costs.shape, dtype=int)
    
    print(f'Поставщики: {suppliers}')
    print(f'Потребители: {consumers}')
    print(f'Матрица тарифов:\n {costs}')

    print(f'Проверка исходных данных на сбалансированность:')
    if np.sum(suppliers) != np.sum(consumers):
        print(f'задача несбалансирована, ∑a = {np.sum(suppliers)}, ∑b = {np.sum(consumers)}')
        return
    print(f'задача сбалансирована, ∑a=∑b={np.sum(suppliers)}')

    return suppliers, consumers, costs, capacity

# Лабораторная работа №1-2
## Разработка программного средства на основе алгоритмов методов нахождения приоритетного решения с использованием количественной оценки (методы угла, метод наименьшего элемента, метод циклических перестановок, метод потенциалов)

### Транспортная задача
Транспортная задача — это математическая задача по нахождению оптимального распределения поставок однородного «товара» (груза, вещества) между пунктами отправления и назначения при заданных, численно выраженных затратах (стоимостях, расходах) на перевозку. Общее решение изначально описано методами линейной алгебры, как для задачи линейного программирования специального вида. Транспортная задача может быть представлена на письме в виде прямоугольной таблицы.

*Пример задачи*:

В $ {m} $ пунктах производства $A_{1}, ..., A_{m}$ находится однородный продукт (сахар, уголь, картофель и т. д.) в количествах соответственно $a_{1}, ..., a_{m}$, который должен быть доставлен $ {n} $ потребителям $B_{1}, ..., B_{n}$ в количествах $ b_{1}, ..., b_{n}$. Известны транспортные издержки $ c_{ij} $ (расходы), связанные с перевозкой единицы продукции из пункта $ A_{i} $ в пункт $ B_{j} $.

Требуется составить такой план перевозок, который обеспечивал бы при минимальных транспортных издержках удовлетворение спроса всех пунктов потребления за счет распределения всего продукта, произведенного всеми пунктами производства.

Для разрешимости поставленной задачи необходимо и достаточно, чтобы сумма запасов равнялась сумме спроса всех пунктов, т. е.
$ \sum_{i=1}^{m} a_{i} = \sum_{j=1}^{n} b_{j} $

Для наглядности транспортную задачу представим в виде таблицы, которая называется распределительной.

![распределительная таблица](./data/table.png)

Здесь количество груза, перевозимого из $ i $-гo пункта отправления в $ j $-й пункт назначения, равно $x_{ij}$, запас груза в $ i $-м пункте отправления измеряется величиной $ a_{i} \geqslant 0 $, а потребность $ j $-го пункта назначения в грузе равна $ b_{j} \geqslant 0 $. Поскольку отрицательные перевозки не имеют реального смысла для данной задачи (обратная перевозка от пунктов назначения в пункты отправления), будем предполагать, что $ x_{ij} \geqslant 0 $.

Матрица $(c_{ij})_{m \times n}$ называется матрицей тарифов (издержек или транспортных расходов), а числа $c_{ij}$ — тарифами. 

Планом транспортной задачи называется матрица $X=(x_{ij})_{m \times n}$ где каждое число $x_{ij}$ обозначает количество единиц груза, которое надо доставить из $i$-го пункта отправления в $j$-й пункт назначения. Матрицу $X$ называют еще матрицей перевозок.

Общие суммарные затраты, связанные с реализацией плана перевозок, можно представить целевой функцией

$f=c_{11}x_{11}+c_{12}x_{12}+ ... +c_{1n}x_{1n}+ ... +c_{m1}x_{m1}+c_{m2}x_{m2}+ ... +c_{mn}x_{mn} = \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij}x_{ij}$

Переменные $x_{ij}$ должны удовлетворять ограничениям по запасам, по потребностям и условиям неотрицательности.

Данные условия образуют систему ограничений. Любой план, компоненты которого удовлетворяют этой системе, будет допустимым.

Система ограничений задачи содержит $m+n$ уравнений с $mn$ переменными $x_{ij}$.

Для транспортной задачи важное значение имеет 
следующая теорема. Ранг матрицы транспортной задачи на единицу меньше числа уравнений, т. е. $r=m+n-1$.

Из теоремы следует, что каждый опорный план имеет $m+n-1$ базисных переменных и $mn-(m+n-1)=(m-1) \times (n-1)$ свободных переменных, равных нулю.

Транспортные задачи будем решать с помощью общего приема последовательного улучшения планов, состоящего из
следующих основных этапов: 
1. определения исходного опорного плана; 
2. оценки этого плана; 
3. перехода к следующему плану путем однократного замещения одной базисной переменной на свободную.

### Метод северо-западного угла
Для составления исходного плана перевозок удобно пользоваться правилом «северо-западного угла», которое состоит в следующем. Будем заполнять таблицу начиная с левого верхнего (северо-западного) угла, двигаясь далее по строке вправо или по столбцу вниз. Занесем в клетку (1, 1) меньшее из чисел $ a_{1} $ и $ b_{1} $ т. е.

$ x_{11}=min(a_{1}; b_{1}) $.

Если $a_{1}>b_{1}$, то $x_{11}=b_{1}$ и первый столбец «закрыт» для заполнения остальных его клеток, т. е. $x_{i1} = 0$ для $i = 2, 3, ..., m$ (потребности первого потребителя удовлетворены полностью).
Двигаясь далее по первой строке, записываем в соседнюю клетку (1,2) меньшее из чисел $a_{1}-b_{1}$ и $b_{2}$, т. е.

$x_{12}=min(a_{1}-b_{1}; b_{2})$.

Если $b_{1} > a_{1}$, то аналогично «закрывается» первая строка, т, е. $x_{1k}=0$ для $k = 2, 3, ..., n$. Переходим к заполнению соседней клетки (2, 1), куда заносим $x_{21}=min(a_{2}; b_{1}-a_{1})$.

Заполнив вторую клетку (1,2) или (2, 1), переходим к заполнению следующей, третьей клетки либо по второй строке, либо по второму столбцу. Будем продолжать этот процесс до полного исчерпания груза у поставщиков или полного удовлетворения потребителей. Последняя заполненная клетка $(m,n)$ окажется лежащей в последнем $n$-м столбце и в последней
$m$-й строке.

После получения опорного плана необходимо проверить его на невырожденность.
Правило: количество базисных (заполненных) клеток в первоначальном плане ВСЕГДА должно быть равно $m + n - 1$, где $m$ - количество поставщиков, $n$ - количество потребителей транспортной задачи.

Что же делать, если количество заполненных ячеек опорного плана меньше необходимого?

На некотором шаге получения первоначального плана может сложиться ситуация, когда одновременно удовлетворяются потребности магазина и опустошается склад.  В этом случае происходит "потеря" базисной клетки. Это приводит к тому, что система определения потенциалов имеет не единственное решение. 

Чтобы обойти эту ситуацию, добавим к базисным ячейкам недостающее количество ячеек с нулевыми значениями. Нулевое значение поставим в клетку, стоящую рядом с базисной клеткой, которая обусловила "пропажу" базисного значения. 

План, полученный по правилу «северо-западного угла», будет опорным планом системы ограничений задачи.

#### Решение задачи
*Входные данные:*

In [14]:
suppliers, consumers, costs, capacity = initProgram('data/Problem5.json')

Поставщики: [23 32 12 15 16 20]
Потребители: [32 11 17 14 21 23]
Матрица тарифов:
 [[ 9  8  4 10  6  9]
 [ 6  9  8  5 11 15]
 [ 9  5 14 11 12  6]
 [14  3  6 10  4  8]
 [ 9  5 10 15  3 12]
 [ 8  4 10 10 12 15]]
Проверка исходных данных на сбалансированность:
задача сбалансирована, ∑a=∑b=118


*Исходный код решения:*

In [ ]:
def northWest(suppliers, consumers, costs, capacity):
    consumer_index = 0
    lost_basis = []
    while consumer_index < len(consumers):
        supplier_index = 0
        while supplier_index < len(suppliers):
            consumer = consumers[consumer_index]
            supplier = suppliers[supplier_index]
            if consumer == 0:
                break
            if supplier == 0:
                supplier_index += 1
                continue
            # проверка "потери" базисной клетки
            if supplier == consumer and supplier_index != len(suppliers)-1 and consumer_index != len(consumers)-1:
                lost_basis.insert(0, (supplier_index, consumer_index+1))
            # выбираем наименьшее количество товара, которое возможно переместить
            capacity[supplier_index][consumer_index] = min(supplier, consumer)
            # вычитаем получившееся количество товара у поставщика и потребителя
            consumers[consumer_index] -= capacity[supplier_index][consumer_index]
            suppliers[supplier_index] -= capacity[supplier_index][consumer_index]
            supplier_index += 1
        consumer_index += 1

    # вычисляем индексы ненулевых значений матрицы перевозок
    a = tuple(map(tuple, np.transpose(np.nonzero(capacity))))

    # вычисляем стоимость перевозок
    sum = 0
    for i in a:
        sum += costs[i]*capacity[i]

    # маскируем все нулевые элементы матрицы перевозок
    capacity = ma.array(capacity)
    capacity = ma.masked_equal(capacity, 0)
    # удаляем маску на "пропавших" базисных элементах
    for i in lost_basis:
        capacity[i] = ma.nomask
    return capacity, sum

*Результат выполнения:*

In [15]:
northWestCap, northWestSum = northWest(suppliers, consumers, costs, capacity)
print(f'Получившийся опорный план:\n{northWestCap}')
print(f'Суммарные затраты перевозок: {northWestSum}')

Получившийся опорный план:
[[23 -- -- -- -- --]
 [9 11 12 -- -- --]
 [-- -- 5 7 -- --]
 [-- -- -- 7 8 --]
 [-- -- -- -- 13 3]
 [-- -- -- -- -- 20]]
Суммарные затраты перевозок: 1080


### Метод наименьшего элемента

Исходный опорный план, построенный по правилу «северо-западного угла», обычно оказывается весьма далеким от оптимального, так как при его определении игнорируются величины затрат $c_{ij}$. Поэтому в дальнейших расчетах потребуется много операций для достижения оптимального плана. Число итераций можно сократить, если исходный план строить по более-усовершенствованному правилу «наименьшего элемента». Сущность его состоит в том, что на каждом шаге осуществляется максимально возможная поставка в клетку с минимальным тарифом $c_{ik}$. Заполнение таблицы начинаем с клетки, которой соответствует наименьший элемент $c_{ik}$ из всей матрицы тарифов. Затем остаток по столбцу или строке помещаем в клетку того же столбца или строки, которой соответствует следующее по величине значение $c_{ik}$ и т. д. Иными словами, последовательность заполняемых клеток определяется по величине $c_{ik}$, а помещаемые в этих
клетках величины $x_{ik}$ как и в правиле «северо-западного угла».

Может оказаться, что при построении опорного плана занятых клеток будет меньше чем $m+n-1$. В этом случае задача называется вырожденной. Тогда в свободную клетку (в ту, которой соответствует наименьший тариф) заносится «базисный» нуль, и эта клетка считается занятой.

#### Решение задачи
*Входные данные:*

In [24]:
suppliers, consumers, costs, capacity = initProgram('data/Problem5.json')

Поставщики: [23 32 12 15 16 20]
Потребители: [32 11 17 14 21 23]
Матрица тарифов:
 [[ 9  8  4 10  6  9]
 [ 6  9  8  5 11 15]
 [ 9  5 14 11 12  6]
 [14  3  6 10  4  8]
 [ 9  5 10 15  3 12]
 [ 8  4 10 10 12 15]]
Проверка исходных данных на сбалансированность:
задача сбалансирована, ∑a=∑b=118


*Исходный код решения:*

In [25]:
def minElem(suppliers, consumers, costs, capacity):
    costs = ma.array(costs)
    # пока все элементы матрицы перевозок не замаскированы
    while costs.mask.all() == False:
        # находим индекс минимального элемента матрицы тарифов
        idx = np.unravel_index(costs.argmin(), costs.shape)
        # выбираем наименьшее количество товара, которое возможно переместить
        if suppliers[idx[0]] < consumers[idx[1]]:
            # записываем его в матрицу перевозок
            capacity[idx] = suppliers[idx[0]]
            # маскируем строку матрицы тарифов
            costs[idx[0]] = ma.masked
        else:
            capacity[idx] = consumers[idx[1]]
            # маскируем столбец матрицы тарифов
            costs[:, idx[1]] = ma.masked
        # вычитаем получившееся количество товара у поставщика и потребителя
        suppliers[idx[0]] -= capacity[idx]
        consumers[idx[1]] -= capacity[idx]

    costs.mask = ma.nomask
    # вычисляем индексы ненулевых значений матрицы перевозок
    a = tuple(map(tuple, np.transpose(np.nonzero(capacity))))
    # вычисляем стоимость перевозок
    sum = 0
    for i in a:
        sum += costs[i]*capacity[i]
    # маскируем все нулевые элементы матрицы перевозок
    capacity = ma.array(capacity)
    capacity = ma.masked_equal(capacity, 0)
    # проверка матрицы перевозок на вырожденность
    if capacity.count() != (len(suppliers)+len(consumers)-1):
        # маскируем элементы матрицы тарифов в соответствии
        # с ненулевыми элементами матрицы перевозок
        for i in a:
            costs[i] = ma.masked
        # добавляем недостающее количество базисных нулей в матрицу перевозок
        for _ in range(len(suppliers)+len(consumers)-1 - capacity.count()):
            # находим индекс минимального элемента матрицы тарифов
            idx = np.unravel_index(costs.argmin(), costs.shape)
            # маскируем этот элемент в матрице тарифов
            costs[idx] = ma.masked
            # удаляем маску этого элемента в матрице перевозок
            capacity[idx] = ma.nomask
    return capacity, sum

*Результат выполнения:*

In [26]:
minElCap, minElSum = minElem(suppliers, consumers, costs, capacity)
print(f'Получившийся опорный план:\n{minElCap}')
print(f'Суммарные затраты перевозок: {minElSum}')

Получившийся опорный план:
[[-- -- 17 -- 1 5]
 [18 -- -- 14 -- --]
 [-- -- -- -- -- 12]
 [-- 11 -- -- 4 --]
 [-- -- -- -- 16 --]
 [14 -- -- -- -- 6]]
Суммарные затраты перевозок: 668


Опорный план и суммарные затраты перевозок методом северо-западного угла:

In [27]:
print(f'Получившийся опорный план:\n{northWestCap}')
print(f'Суммарные затраты перевозок: {northWestSum}')

Получившийся опорный план:
[[23 -- -- -- -- --]
 [9 11 12 -- -- --]
 [-- -- 5 7 -- --]
 [-- -- -- 7 8 --]
 [-- -- -- -- 13 3]
 [-- -- -- -- -- 20]]
Суммарные затраты перевозок: 1080


### Метод потенциалов

Сущность метода потенциалов состоит в следующем. После того, как найден исходный опорный план перевозок, каждому поставщику $A_{i}$ (каждой строке) ставится в соответствие некоторое число $u_{i} (i=1, m)$, а каждому потребителю $B_{j}$ (каждому столбцу) — некоторое число $v_{j}$. Числа $u_{i}$ и $v_{j}$ называются потенциалами соответственно поставщика $A_{i}$ и потребителя $B_{j}$ и выбираются так, чтобы в любой загруженной клетке их сумма равнялась тарифу этой клетки, т. е. $u_{i}+v_{j}=c_{ij}$. Так как всех потенциалов $m+n$, а занятых клеток $m+n-1$, то для определения чисел $u_{i}$ и $v_{j}$ придется решать систему из $m+n-1$ уравнений $u_{i}+v_{j}=c_{ij}$ с $m+n$ неизвестными. В этом случае одной из неизвестных можно придать произвольное значение, и тогда система будет иметь единственное решение, т. е. все остальные $m+n-1$ неизвестных определятся однозначно.Затем для проверки оптимальности плана просматриваются свободные клетки $(i, j)$ и для каждой из них вычисляется разность $s_{ij}$ между тарифом $c_{ij}$ и суммой $u_{i}+v_{j}$ потенциалов строки и столбца. План оптимален, когда для каждой свободной клетки $(i, j)$ разность $s_{ij}$ есть величина неотрицательная, т. е. $s_{ij}=c_{ij}-(u_{i}+v_{j}) \geqslant 0$.

Полученные разности называются оценками (характеристиками) свободных клеток. Отрицательные оценки указывают на перспективность клеток, загрузка их приведет к улучшению плана. Положительные и нулевые оценки исключают возможность улучшения полученного плана. Переход к новому плану осуществляется по общим правилам распределительного
метода. Для наиболее перспективной клетки строится замкнутый контур, вершинам которого приписываются чередующиеся знаки (свободной клетке приписывается положительный знак). В клетках, соответствующих отрицательным вершинам, отыскивается наименьший груз, который и «перемещается» по клеткам замкнутого цикла, т. е. прибавляется к клеткам со знаком плюс, включая свободную, и вычитается из клеток со знаком минус.

В новом плане вновь определяются потенциалы строк и столбцов и вычисляются оценки для всех свободных клеток. Когда среди оценок не окажется отрицательных, полученный план будет оптимальным.

Итак, чтобы решить транспортную задачу методом потенциалов, необходимо:
1. построить опорный план перевозок по одному из вышеизложенных правил;
2. вычислить потенциалы $u_{i}$ и $v_{j}$ соответственно поставщиков и потребителей;
3. вычислить суммы потенциалов (косвенные тарифы) для свободных клеток $u_{i}+v_{j}=c'_{ij}$;
4. проверить разность $s_{ij}=c_{ij}-c'_{ij}$.

Если все $s_{ij} \geqslant 0$ для свободных клеток, полученный план оптимален. Если хотя бы одна оценка $s_{ij} < 0$, в число занятых вводят клетку, для которой оценка минимальна, и получают новый план перевозок. Процесс продолжают до тех пор, пока не будет получен план, для которого все оценки $s_{ij} \geqslant 0$.

Алгебраическая сумма $s_{ij}$ стоимостей по циклу пересчета свободной клетки $(i, j)$ равна разности между стоимостью $c_{ij}$ и суммой потенциалов $u_{i}$ и $v_{j}$, т. е.

$s_{ij}=c_{ij}-c'_{ij}=c_{ij}-(u_{i}+v_{j})$.

#### Решение задачи

Возьмем опорные планы метода северо-западного угла и метода наименьшего элемента.

*Входные данные:*

In [32]:
suppliers, consumers, costs, capacity = initProgram('data/Problem5.json')
print(f'Опорный план метода северо-западного угла:\n{northWestCap}')
print(f'Суммарные затраты перевозок: {northWestSum}')
print(f'Опорный план метода наименьшего элемента:\n{minElCap}')
print(f'Суммарные затраты перевозок: {minElSum}')

Поставщики: [23 32 12 15 16 20]
Потребители: [32 11 17 14 21 23]
Матрица тарифов:
 [[ 9  8  4 10  6  9]
 [ 6  9  8  5 11 15]
 [ 9  5 14 11 12  6]
 [14  3  6 10  4  8]
 [ 9  5 10 15  3 12]
 [ 8  4 10 10 12 15]]
Проверка исходных данных на сбалансированность:
задача сбалансирована, ∑a=∑b=118
Опорный план метода северо-западного угла:
[[23 -- -- -- -- --]
 [9 11 12 -- -- --]
 [-- -- 5 7 -- --]
 [-- -- -- 7 8 --]
 [-- -- -- -- 13 3]
 [-- -- -- -- -- 20]]
Суммарные затраты перевозок: 1080
Опорный план метода наименьшего элемента:
[[-- -- 17 -- 1 5]
 [18 -- -- 14 -- --]
 [-- -- -- -- -- 12]
 [-- 11 -- -- 4 --]
 [-- -- -- -- 16 --]
 [14 -- -- -- -- 6]]
Суммарные затраты перевозок: 668


*Исходный код решения:*

In [35]:
def potential(costs, mask):
    u = ma.masked_all(costs.shape[0], dtype=int)
    v = ma.masked_all(costs.shape[1], dtype=int)
    # предположим, первый элемент матрицы u равен нулю
    u[0] = 0

    # вычисляем потенциалы u и v
    while True:
        i = 0
        j = 0
        # проходим по матрице стоимостей
        for row in mask:
            for el in row:
                # если стоимость замаскирована, невозможно вычислить u или v
                if el == True:
                    j += 1
                    continue
                # вычисляем u или v, если они еще не вычислены
                if ma.is_masked(u[i]) and not ma.is_masked(v[j]):
                    u[i] = costs[i][j] - v[j]
                if ma.is_masked(v[j]) and not ma.is_masked(u[i]):
                    v[j] = costs[i][j] - u[i]
                j += 1
            j = 0
            i += 1
        # проверяем, все ли значения массивов u и v вычислены
        if not ma.is_masked(u) and not ma.is_masked(v):
            break

    # вычисляем косвенные тарифы
    indCosts = costs.copy()
    i = 0
    j = 0
    for row in mask:
        for el in row:
            if el == True:
                # s = c - c'; c' = u + v
                indCosts[i][j] = costs[i][j] - (u[i]+v[j])
            j += 1
        j = 0
        i += 1

    # вычисляем индексы отрицательных значений матрицы косвенных тарифов
    costsIndex = tuple(map(tuple, np.transpose(np.where(indCosts < 0))))

    return u, v, indCosts, costsIndex

*Результат выполнения:*

In [41]:
u, v, inderectCosts, costsIndex = potential(costs, ma.getmask(northWestCap))
print(f'Метод северо-западного угла\nПотенциал поставщика u:{u}\nПотенциал потребителя v:{v}\nКосвенные тарифы:\n{inderectCosts}\nПотенциальные клетки для загрузки (с отрицательными косвенными тарифами):{costsIndex}\n')
u, v, inderectCosts, costsIndex = potential(costs, ma.getmask(minElCap))
print(f'Метод минимального элемента\nПотенциал поставщика u:{u}\nПотенциал потребителя v:{v}\nКосвенные тарифы:\n{inderectCosts}\nПотенциальные клетки для загрузки (с отрицательными косвенными тарифами):{costsIndex}\n')

Метод северо-западного угла
Потенциал поставщика u:[0 -3 3 2 1 4]
Потенциал потребителя v:[9 12 11 8 2 11]
Косвенные тарифы:
[[  9  -4  -7   2   4  -2]
 [  6   9   8   0  12   7]
 [ -3 -10  14  11   7  -8]
 [  3 -11  -7  10   4  -5]
 [ -1  -8  -2   6   3  12]
 [ -5 -12  -5  -2   6  15]]
Потенциальные клетки для загрузки (с отрицательными косвенными тарифами):((0, 1), (0, 2), (0, 5), (2, 0), (2, 1), (2, 5), (3, 1), (3, 2), (3, 5), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2), (5, 3))

Метод минимального элемента
Потенциал поставщика u:[0 4 -3 -2 -3 6]
Потенциал потребителя v:[2 5 4 1 6 9]
Косвенные тарифы:
[[ 7  3  4  9  6  9]
 [ 6  0  0  5  1  2]
 [10  3 13 13  9  6]
 [14  3  4 11  4  1]
 [10  3  9 17  3  6]
 [ 8 -7  0  3  0 15]]
Потенциальные клетки для загрузки (с отрицательными косвенными тарифами):((5, 1),)

